① Tool Definitions + Messages
개발자가 사용할 수 있는 함수(예: get_weather(location))를 미리 정의
사용자가 질문: “What’s the weather in Paris?”

② Tool Calls
모델이 질문을 보고 “아, 이건 get_weather("paris") 함수를 호출해야겠네”
텍스트가 아니라 함수 호출 요청(JSON) 을 생성

③ Execute Function Code
실제 코드에서 get_weather("paris") 실행
외부 API(OpenWeather 같은) 호출

④ Results (All Prior Messages)
함수 실행 결과가 다시 모델에게 전달
모델은 이제 “파리의 온도 = 14도”라는 사실을 알게 됨

⑤ Final Response
모델이 사용자에게 자연어로 최종 답변 생성
“It’s currently 14°C in Paris.”

- LLM이 API를 직접 실행하는 게 아니라
“어떤 함수를 호출할지 결정”만 하고
실행은 개발자 코드,
결과를 다시 받아 문장 생성
👉 LLM + 외부 시스템 연동 구조

In [1]:
from openai import OpenAI
from dotenv import load_dotenv                                                # .env 파일의 환경변수 로드
import os                                                                     # 환경변수 접근용

load_dotenv()                                                                 # 현재 위치의 .env를 읽어와 환경변수로 등록
os.environ['OPENAI_API_KEY']=os.getenv("openai_key")                          # .env의 openai_key 값을 가져옴
OPENWEATHER_API_KEY = os.getenv('openweather_key')                            # .env의 openweather_key 값을 가져옴

In [2]:
# OpenWeather API 호출로 부산 날씨 데이터 조회
import requests

city_name = "Busan"         # 조회할 도시
units = 'metric'            # 단위 (metric = 섭씨)
# Openweather 현재 날씨 API URL 생성
url = f"https://api.openweathermap.org/data/2.5/weather?q={city_name}&appid={OPENWEATHER_API_KEY}&units={units}"
response = requests.get(url) # API 요청 전송
data = response.json()       # 
data

{'coord': {'lon': 129.0403, 'lat': 35.1028},
 'weather': [{'id': 800,
   'main': 'Clear',
   'description': 'clear sky',
   'icon': '01d'}],
 'base': 'stations',
 'main': {'temp': -2.01,
  'feels_like': -6.44,
  'temp_min': -2.01,
  'temp_max': -2.01,
  'pressure': 1030,
  'humidity': 43,
  'sea_level': 1030,
  'grnd_level': 1025},
 'visibility': 10000,
 'wind': {'speed': 3.6, 'deg': 310},
 'clouds': {'all': 0},
 'dt': 1770079552,
 'sys': {'type': 1,
  'id': 8086,
  'country': 'KR',
  'sunrise': 1770070906,
  'sunset': 1770108802},
 'timezone': 32400,
 'id': 1838524,
 'name': 'Busan',
 'cod': 200}

In [3]:
weather_info =  {}      # 최종 결과를 담을 딕셔너리

if response.status_code == 200:   # 응답 코드 200이면 정상 처리되었다.
    weather_description = data['weather'][0]['description'] # 날씨 설명
    temp = data['main']['temp']                             # 온도
    temp_feels_like = data['main']['feels_like']            # 체감 온도
    humidity = data['main']['humidity']                     # 습도
    

    weather_info = {
        'city' : city_name,
        'description' : weather_description,
        'temperature' : temp,
        'temperature_feels_like' : temp_feels_like,
        'humidity' : humidity    

}

else:
    weather_info = {
    'city' : city_name,
    'description' : 'Not Found',
    'temperature' : 'Not Found',
    'temperature_feels_like' : 'Not Found',
    'humidity' : 'Not Found'  
    }

weather_info


{'city': 'Busan',
 'description': 'clear sky',
 'temperature': -2.01,
 'temperature_feels_like': -6.44,
 'humidity': 43}

In [4]:
# 실패시 실패 원인 확인용 디버깅

print("status_code: ", response.status_code)                # HTTP 상태코드(200=성공, 401=키 문제 등)
print("response json :", data)                              # 응답받은 JSON
print("message :", data.get("message", "no message"))       # 에러 메시지 키

status_code:  200
response json : {'coord': {'lon': 129.0403, 'lat': 35.1028}, 'weather': [{'id': 800, 'main': 'Clear', 'description': 'clear sky', 'icon': '01d'}], 'base': 'stations', 'main': {'temp': -2.01, 'feels_like': -6.44, 'temp_min': -2.01, 'temp_max': -2.01, 'pressure': 1030, 'humidity': 43, 'sea_level': 1030, 'grnd_level': 1025}, 'visibility': 10000, 'wind': {'speed': 3.6, 'deg': 310}, 'clouds': {'all': 0}, 'dt': 1770079552, 'sys': {'type': 1, 'id': 8086, 'country': 'KR', 'sunrise': 1770070906, 'sunset': 1770108802}, 'timezone': 32400, 'id': 1838524, 'name': 'Busan', 'cod': 200}
message : no message


In [5]:
# 날씨 API 호출 함수: doc_string을 활용해 함수 설명
import requests
import json

def get_current_weather(city_name='Seoul', units='metric'):
    """  OpenWeather API를 사용해서 사용자가 지정한 도시의 현재 날씨 정보를 가져오는 함수
    Args:
        city: str 날씨정보를 가져올 도시 이름. 반드시 영문으로 작성하세요.
            변환예시:
                서울 -> Seoul
                충남, 충청남도 -> Chungcheongnam-do
                부산 -> Busan
        units: str 온도단위를 설정하는 문자열
            metric(기본값: 섭씨, 미터)
            imperial(화씨, 야드)
    Return:
            str: json 형식으로 변환된 현재 날씨 정보
    """
    url = f"https://api.openweathermap.org/data/2.5/weather?q={city_name}&appid={OPENWEATHER_API_KEY}&units={units}"
    response = requests.get(url) # API 요청 전송
    data = response.json()   

    if response.status_code == 200:   # 응답 코드 200이면 정상 처리되었다.
        weather_description = data['weather'][0]['description'] # 날씨 설명
        temp = data['main']['temp']                             # 온도
        temp_feels_like = data['main']['feels_like']            # 체감 온도
        humidity = data['main']['humidity']                     # 습도
        

        weather_info = {
            'city' : city_name,
            'description' : weather_description,
            'temperature' : temp,
            'temperature_feels_like' : temp_feels_like,
            'humidity' : humidity    

    }

    else:
        weather_info = {
        'city' : city_name,
        'description' : 'Not Found',
        'temperature' : 'Not Found',
        'temperature_feels_like' : 'Not Found',
        'humidity' : 'Not Found'  
        }

    return json.dumps(weather_info)  #dict를 JSON 문자열로 반환해 반환



In [6]:
# LLM이 사용할 함수(tool)

tools_to_execute = {
    'get_current_weather'  : get_current_weather,   # 키(key): LLM이 호출할 tool 이름, 값(value) : 실제 실행될 함수
}

## LLM 준비

In [7]:
# get_current_weather 함수의 docstring(설명) 출력
print(tools_to_execute['get_current_weather'].__doc__)

  OpenWeather API를 사용해서 사용자가 지정한 도시의 현재 날씨 정보를 가져오는 함수
    Args:
        city: str 날씨정보를 가져올 도시 이름. 반드시 영문으로 작성하세요.
            변환예시:
                서울 -> Seoul
                충남, 충청남도 -> Chungcheongnam-do
                부산 -> Busan
        units: str 온도단위를 설정하는 문자열
            metric(기본값: 섭씨, 미터)
            imperial(화씨, 야드)
    Return:
            str: json 형식으로 변환된 현재 날씨 정보
    


In [8]:
# Function Calling으로 날씨 API를 호출해 답변하는 대화 루프  # 전체 목적: 유저 질문 → 필요하면 함수 호출 → 최종 답변 생성
from pprint import pprint                                  # messages 같은 자료구조를 보기 좋게 출력하는 도구
from openai import OpenAI                                  # OpenAI 파이썬 SDK의 클라이언트 클래스
import json

client = OpenAI()                                          # OpenAI API 호출용 클라이언트 생성

# 사용자 질문을 받아 tool 호출 여부를 판단하고 최종 답변까지 생성하는 함수
def run_conversation(user_prompt, model="gpt-4.1-mini"):    # user_prompt: 유저 입력 / model: 사용할 모델 이름(기본값 gpt-4.1-mini)
    messages = [                                            # 대화 기록(모델에 그대로 전달됨). 리스트 안에 role/content 딕셔너리들
        {
            "role": "system",                               # system: 모델의 행동 규칙/지침을 설정하는 역할
            "content": (                                    # system 프롬프트(모델이 지켜야 할 “캐릭터/규칙”)
                "당신은 친절한 챗봇입니다. 사용자의 요구를 분석해 직접 대답하거나, "
                "주어진 함수를 이용해 필요한 정보를 먼저 확보한 후 대답하세요."
            ),
        },
        {
            "role": "user",                                 # user: 실제 사용자의 질문 역할
            "content": user_prompt,                         # 유저 프롬프트(함수 인자로 들어온 user_prompt를 그대로 넣음)
        },
    ]

    # 모델이 호출할 수 있는 함수 목록(툴 스키마)                     # 여기부터 tools 정의
    tools = [
        {
            "type": "function",                             # tool 종류: function(함수 호출)
            "function": {
                "name": "get_current_weather",             # 모델이 호출할 함수 이름(툴 이름)
                "description": tools_to_execute["get_current_weather"].__doc__,  # 함수 설명(보통 docstring을 넣음) 
                "parameters": {                             # 함수 인자 스키마(JSON Schema 형태)
                    "type": "object",                       # 인자는 객체 형태로 받음 (예: {"city_name": "...", "units": "..."} )
                    "properties": {                         # 어떤 키들을 받을 수 있는지 정의
                        "city_name": {
                            "type": "string",               # city_name은 문자열
                            "description": (                # 모델에게 “도시이름은 영어로!” 같은 규칙을 알려주는 설명
                                "도시이름(필수값). 반드시 영어로 작성하세요.\n"
                                "변환예시:\n"
                                "  서울 -> Seoul\n"
                                "  충남, 충청남도 -> Chungcheongnam-do\n"
                                "  경남, 경상남도 -> Gyeongsangnam-do\n"
                                "  전남, 전라남도 -> Jeollanam-do\n"
                                "  부산 -> Busan"
                            ),
                        },
                        "units": {
                            "type": "string",               # units도 문자열
                            "description": (                # 온도 단위를 어떤 값으로 할지 설명
                                "온도단위를 설정하는 문자열\n\n"
                                "metric(기본값: 섭씨, 미터)\n"
                                "imperial(화씨, 야드)"
                            ),
                            "enum": ["metric", "imperial"], # 허용 값(이 외에는 모델이 선택 못하게 제한)
                        },
                    },
                    "required": ["city_name"],              # 필수 인자: city_name은 무조건 있어야 함(units는 선택)
                }
            }
        }
    ]

    response1 = client.chat.completions.create(             # 1차 호출: 모델이 답변하거나(tool 호출) 둘 중 하나를 선택
        model=model,                                        # 사용할 모델명
        messages=messages,                                  # 지금까지의 대화 기록
        tools=tools                                         # 모델에게 “이 함수 호출 가능해”라고 알려줌
    )
    
    response1_message = response1.choices[0].message        # 모델이 준 첫 번째 응답 메시지 객체
    response1_tool_calls = response1_message.tool_calls     # 모델이 tool 호출을 했는지 확인(했으면 여기에 들어있음)

    if response1_tool_calls:                                # tool_calls가 있으면: 모델이 함수를 호출하겠다고 한 상태
        messages.append(response1_message)                  # 모델이 “이 함수 호출해줘”라고 요청한 메시지도 대화 기록에 추가

        for tool_call in response1_tool_calls:              # tool 호출이 여러 개일 수 있어서 반복
            function_name = tool_call.function.name         # 호출할 함수 이름 (예: get_current_weather)
            print(f"[tool] {function_name}을 호출합니다!")  # 디버깅용 출력(실제로 함수 실행하는지 확인)

            function_to_execute = tools_to_execute[function_name]  # 미리 준비한 파이썬 함수 dict에서 꺼냄
            
            function_args = json.loads(tool_call.function.arguments) #  JSON 문자열 파싱

            function_response = function_to_execute(**function_args) # 파싱된 인자를 함수에 **로 풀어서 전달

            messages.append({                               # tool 실행 결과를 모델에게 다시 전달하기 위한 메시지
                "role": "tool",                             # role=tool: “이건 도구 실행 결과야”
                "tool_call_id": tool_call.id,               # 어떤 tool_call에 대한 응답인지 연결하는 id
                "name": function_name,                      # 실행한 함수 이름
                "content": function_response                # 함수 실행 결과(문자열/JSON 문자열 등)
            })

            pprint(messages)                                # 현재까지 messages를 보기 좋게 출력(디버깅)

        response2 = client.chat.completions.create(         # 2차 호출: tool 결과까지 포함한 messages로 최종 답변 생성
            model=model,                                    # 같은 모델 사용
            messages=messages                               # tool 결과가 포함된 대화 기록
        )
        return response2.choices[0].message.content         # 최종 답변 텍스트 반환
    
    else:                                                   # tool_calls가 없으면: 모델이 바로 답변을 준 경우
        return response1_message.content                    # 1차 응답 내용 그대로 반환


In [12]:
run_conversation("오늘 뉴욕은 어때?")

[tool] get_current_weather을 호출합니다!
[{'content': '당신은 친절한 챗봇입니다. 사용자의 요구를 분석해 직접 대답하거나, 주어진 함수를 이용해 필요한 정보를 먼저 확보한 '
             '후 대답하세요.',
  'role': 'system'},
 {'content': '오늘 뉴욕은 어때?', 'role': 'user'},
 ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageFunctionToolCall(id='call_cRuXS0AEPBwGHplZShcXSug1', function=Function(arguments='{"city_name":"New York"}', name='get_current_weather'), type='function')]),
 {'content': '{"city": "New York", "description": "clear sky", "temperature": '
             '-0.99, "temperature_feels_like": -5.93, "humidity": 43}',
  'name': 'get_current_weather',
  'role': 'tool',
  'tool_call_id': 'call_cRuXS0AEPBwGHplZShcXSug1'}]


'오늘 뉴욕은 맑은 하늘이며, 기온은 약 -1도, 체감 온도는 약 -6도입니다. 습도는 43%입니다. 외출 시 따뜻하게 입으시는 것이 좋겠습니다.'